In [ ]:
# Libraries
import googlemaps
import uuid
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
from langchain.agents import create_agent, AgentState
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, Literal, Optional, Dict
from tavily import TavilyClient
from pydantic import BaseModel, Field
from datetime import datetime
from langgraph.graph import StateGraph, START, END

from typing import Any
from langchain.agents.middleware import before_model, ModelRequest
from langchain.messages import SystemMessage

from src.config.settings import settings
from serpapi import GoogleSearch


class AttractionFindingTool:
    def __init__(self):
        self.serapi = settings
    def execute(self, place_name):
        return self._extract_places(place_name)
    def _extract_places(self, place_name):
        
        try:
            params = {
                "engine": "tripadvisor",
                "q": place_name,
                "ssrc": "A",
                "api_key": ""
                }
            search = GoogleSearch(params)
            results = self._parse_data(search.get_dict())
            return results
        except Exception as e:
            return {
                "status": "Error",
                "place_name": place_name,
                "Error": str(e)
            }
    def _parse_data(self, results, place_name):
        attraction_details = []
        attraction_places = results.get("places", [])
        if not attraction_places:
            return {
                "status": "Failed",
                "place_name":place_name,
                "attraction_details": attraction_details,
                "result": "No Attraction Places Found"
            }
        for num in range(5):
            attraction = attraction_places[num]
            
            attraction_details.append(
                {
                    "title": attraction.get('title', []),
                    "description": self.clean_text(attraction.get('description', []),max_chars = 150),
                    "rating": attraction.get('rating', []),
                    "highlighted_review": self.clean_text(attraction.get('highlighted_review', []),max_chars = 150)
                }
            )
        return {
            "status": "Success",
            "place_name": place_name,
            "attraction_details": attraction_details,
            "result": "Attraction Places Found"
        }
    def clean_text(self, text_data, max_chars=150):
        """
        1. Extracts text if input is a dict (for highlighted_review).
        2. Truncates text to max_chars.
        3. Ensures no words are cut in half.
        """
        # Handle case where review is a dictionary {'text': '...', ...}
        if isinstance(text_data, dict):
            text = text_data.get('text', '')
        else:
            text = str(text_data) if text_data else ""

        # Return default if empty
        if not text:
            return "No details available."

        # If it's short enough, return it
        if len(text) <= max_chars:
            return text

        # Truncate and add ellipsis, trying to cut at a space
        truncated = text[:max_chars]
        if " " in truncated:
            truncated = truncated.rsplit(" ", 1)[0] # Cut at last space
        
        return truncated + "..."

In [ ]:
import json
from datetime import datetime
from typing import Optional, Dict, List
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.storage import InMemoryStore
import uuid

# ============================================================================
# 1. IN-MEMORY STORAGE MODULE (Using LangChain InMemoryStore)
# ============================================================================

@dataclass
class UserProfile:
    user_id: str
    created_at: str
    name: Optional[str] = None
    email: Optional[str] = None
    preferences: Dict = None

class SimpleInMemoryStorage:
    """Simple in-memory storage using LangChain's InMemoryStore"""
    
    def __init__(self):
        self.store = InMemoryStore()
        self.users = {}  # {user_id: UserProfile}
        self.conversations = {}  # {user_id: [messages]}
        self.trip_plans = {}  # {plan_id: trip_details}
    
    def create_or_get_user(self, user_id: str, name: str = None) -> UserProfile:
        """Create new user or retrieve existing"""
        if user_id in self.users:
            return self.users[user_id]
        
        now = datetime.now().isoformat()
        user = UserProfile(
            user_id=user_id,
            name=name,
            created_at=now,
            preferences={}
        )
        self.users[user_id] = user
        self.conversations[user_id] = []
        return user
    
    def save_conversation(self, user_id: str, role: str, content: str):
        """Save message to conversation history"""
        if user_id not in self.conversations:
            self.conversations[user_id] = []
        
        message = {
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        }
        self.conversations[user_id].append(message)
        
        # Also store in InMemoryStore
        key = f"conversation_{user_id}_{len(self.conversations[user_id])}"
        self.store.mset([(key, json.dumps(message))])
    
    def get_conversation_history(self, user_id: str, limit: int = 20) -> List[Dict]:
        """Retrieve recent conversation history"""
        if user_id not in self.conversations:
            return []
        
        messages = self.conversations[user_id][-limit:]
        return [{"role": m["role"], "content": m["content"]} for m in messages]
    
    def save_trip_plan(self, user_id: str, destination: str, start_date: str, 
                       duration_days: int, budget: float, interests: List[str], 
                       plan_details: str):
        """Save completed trip plan"""
        plan_id = str(uuid.uuid4())
        
        trip = {
            "plan_id": plan_id,
            "user_id": user_id,
            "destination": destination,
            "start_date": start_date,
            "duration_days": duration_days,
            "budget": budget,
            "interests": interests,
            "plan_details": plan_details,
            "created_at": datetime.now().isoformat()
        }
        
        self.trip_plans[plan_id] = trip
        
        # Store in InMemoryStore
        self.store.mset([(f"trip_{plan_id}", json.dumps(trip))])
        
        return plan_id
    
    def get_user_trip_history(self, user_id: str) -> List[Dict]:
        """Get all previous trip plans for a user"""
        user_trips = [trip for trip in self.trip_plans.values() 
                      if trip["user_id"] == user_id]
        
        # Sort by created_at (most recent first)
        user_trips.sort(key=lambda x: x["created_at"], reverse=True)
        
        return [
            {
                "destination": trip["destination"],
                "start_date": trip["start_date"],
                "duration_days": trip["duration_days"],
                "budget": trip["budget"],
                "interests": trip["interests"],
                "created_at": trip["created_at"]
            }
            for trip in user_trips
        ]
    
    def update_user_preferences(self, user_id: str, preferences: Dict):
        """Update user travel preferences"""
        if user_id in self.users:
            self.users[user_id].preferences = preferences
            self.store.mset([(f"user_prefs_{user_id}", json.dumps(preferences))])


# ============================================================================
# 2. CONVERSATION MANAGER AGENT (Gathers Trip Information)
# ============================================================================

@dataclass
class TripRequirements:
    destination: Optional[str] = None
    start_date: Optional[str] = None
    duration_days: Optional[int] = None
    budget: Optional[float] = None
    interests: List[str] = None
    
    def is_complete(self) -> bool:
        return all([self.destination, self.start_date, self.duration_days, 
                    self.budget, self.interests])
    
    def missing_fields(self) -> List[str]:
        missing = []
        if not self.destination:
            missing.append("destination")
        if not self.start_date:
            missing.append("start_date")
        if not self.duration_days:
            missing.append("duration_days")
        if not self.budget:
            missing.append("budget")
        if not self.interests or len(self.interests) == 0:
            missing.append("interests")
        return missing


class ConversationManager:
    """Manages the interactive question-answer flow to gather trip information"""
    
    def __init__(self, model, storage_manager: SimpleInMemoryStorage):
        self.model = model
        self.storage = storage_manager
        self.requirements = TripRequirements()
        
        self.conversation_prompt = """You are a friendly Travel Planning Assistant. Your role is to gather information from the user to create a personalized trip plan.

INFORMATION NEEDED:
1. Destination: Where do they want to go?
2. Start Date: When do they plan to travel? (format: YYYY-MM-DD)
3. Duration: How many days? (integer)
4. Budget: What's their budget in USD? (number)
5. Interests: What are their travel interests? (list: adventure, culture, food, beaches, history, shopping, nature, nightlife, etc.)

CONVERSATION RULES:
- Ask ONE question at a time.
- Be conversational and friendly.
- If the user provides multiple pieces of info at once, extract and confirm them all.
- Once you have all 5 pieces, respond ONLY with: "INFORMATION_COMPLETE: {json of all details}"
- If user mentions a previous trip, acknowledge it and continue gathering info.
- Be flexible with date formats and convert them to YYYY-MM-DD.

EXAMPLE FORMAT when complete:
INFORMATION_COMPLETE: {
    "destination": "Thailand",
    "start_date": "2026-03-15",
    "duration_days": 7,
    "budget": 2000,
    "interests": ["beaches", "food", "culture"]
}
"""
    
    def process_user_input(self, user_id: str, user_message: str) -> str:
        """Process user input and extract trip information"""
        from langchain.agents import create_agent
        
        # Create a single-turn agent to extract info from the message
        extraction_prompt = f"""You are extracting travel information from user messages.

Current known information:
- Destination: {self.requirements.destination}
- Start Date: {self.requirements.start_date}
- Duration: {self.requirements.duration_days}
- Budget: {self.requirements.budget}
- Interests: {self.requirements.interests}

User message: "{user_message}"

Extract any new information and respond in JSON format:
{{
    "extracted": {{
        "destination": "string or null",
        "start_date": "YYYY-MM-DD or null",
        "duration_days": "integer or null",
        "budget": "float or null",
        "interests": ["array of interests or empty"]
    }},
    "response": "Your friendly conversational response to ask for missing info"
}}

If all 5 fields are now complete, set "response" to: "INFORMATION_COMPLETE"
"""
        
        # Call the model to extract information
        response = self.model.invoke(extraction_prompt)
        
        # Parse the response
        try:
            # Extract JSON from response
            import re
            json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
            if json_match:
                data = json.loads(json_match.group())
                extracted = data.get("extracted", {})
                
                # Update requirements
                if extracted.get("destination"):
                    self.requirements.destination = extracted["destination"]
                if extracted.get("start_date"):
                    self.requirements.start_date = extracted["start_date"]
                if extracted.get("duration_days"):
                    self.requirements.duration_days = int(extracted["duration_days"])
                if extracted.get("budget"):
                    self.requirements.budget = float(extracted["budget"])
                if extracted.get("interests"):
                    self.requirements.interests = extracted["interests"]
                
                # Save to conversation history
                self.storage.save_conversation(user_id, "user", user_message)
                response_text = data.get("response", "Tell me more about your trip!")
                self.storage.save_conversation(user_id, "assistant", response_text)
                
                return response_text
        
        except json.JSONDecodeError:
            return "I had trouble understanding that. Could you provide more details?"
        
        return "Let me help you plan your trip! First, where would you like to go?"
    
    def get_next_question(self) -> str:
        """Get the next question based on missing information"""
        missing = self.requirements.missing_fields()
        
        if not missing:
            return "Great! I have all the information I need to create your personalized trip plan."
        
        question_map = {
            "destination": "🌍 Where would you like to travel? (e.g., Thailand, Japan, Egypt)",
            "start_date": "📅 When are you planning to go? (e.g., 2026-03-15 or March 15, 2026)",
            "duration_days": "⏱️ How many days are you planning to stay?",
            "budget": "💰 What's your total budget in USD? (e.g., 2000 for accommodation, flights, food)",
            "interests": "🎯 What are your travel interests? (e.g., adventure, food, culture, beaches, history, shopping)"
        }
        
        return question_map.get(missing[0], "Tell me more about your trip!")


# ============================================================================
# 3. MAIN TRAVEL PLANNING SYSTEM (Updated)
# ============================================================================

def initialize_system():
    """Initialize all components"""
    import os
    
    # Initialize API keys (from your original code)
    os.environ["GPLACES_API_KEY"] = "YOUR_API_KEY"
    os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"
    os.environ["TAVILY_API_KEY"] = "YOUR_API_KEY"
    
    model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    parent_model = ChatGoogleGenerativeAI(model="gemini-3-pro-preview", temperature=0)
    
    storage = SimpleInMemoryStorage()
    conversation_manager = ConversationManager(model, storage)
    
    return model, parent_model, storage, conversation_manager


class TravelPlanningSession:
    """Main session handler for interactive trip planning"""
    
    def __init__(self, user_id: str, storage: SimpleInMemoryStorage, 
                 conversation_manager: ConversationManager):
        self.user_id = user_id
        self.storage = storage
        self.conversation_manager = conversation_manager
        self.user = storage.create_or_get_user(user_id)
    
    def start_session(self) -> str:
        """Start a new planning session"""
        greeting = f"👋 Hello {self.user.name or 'Traveler'}! Welcome to TravelPartner AI.\n\n"
        greeting += "I'll help you create a personalized trip plan. Let me ask you a few questions.\n\n"
        
        # Check if user has previous trips
        history = self.storage.get_user_trip_history(self.user_id)
        if history:
            greeting += f"📍 I see you've planned {len(history)} trip(s) before. Exciting!\n\n"
        
        # Ask first question
        first_question = self.conversation_manager.get_next_question()
        greeting += first_question
        
        return greeting
    
    def process_message(self, user_message: str) -> str:
        """Process a single user message"""
        # Save to history
        self.storage.save_conversation(self.user_id, "user", user_message)
        
        # Extract information from message
        response = self.conversation_manager.process_user_input(self.user_id, user_message)
        
        # Check if all information is gathered
        if self.conversation_manager.requirements.is_complete():
            response += "\n\n✅ Perfect! I now have all the details. Creating your personalized travel plan..."
            return response
        else:
            # Ask next question
            missing = self.conversation_manager.requirements.missing_fields()
            response += f"\n\n" + self.conversation_manager.get_next_question()
            return response
    
    def generate_plan(self, travel_partner_agent) -> str:
        """Generate the final trip plan once all info is gathered"""
        req = self.conversation_manager.requirements
        
        query = f"""
        I want to go to {req.destination} from Pakistan for {req.duration_days} days starting {req.start_date}.
        My budget is ${req.budget}.
        My interests include: {', '.join(req.interests)}.
        
        Please create a complete personalized plan including:
        - Flight options and recommendations
        - Accommodation suggestions
        - Day-by-day itinerary based on my interests
        - Restaurant recommendations for my dietary preferences
        - Travel tips and local insights
        - Estimated budget breakdown
        """
        
        # Call the main travel planning agent
        result = travel_partner_agent.invoke({"messages": [{"role": "user", "content": query}]})
        
        plan_text = result["messages"][-1].content[0]["text"]
        
        # Save the plan
        self.storage.save_trip_plan(
            self.user_id,
            req.destination,
            req.start_date,
            req.duration_days,
            req.budget,
            req.interests,
            plan_text
        )
        
        return plan_text


# ============================================================================
# 4. USAGE EXAMPLE
# ============================================================================

if __name__ == "__main__":
    # Initialize system
    model, parent_model, storage, conversation_manager = initialize_system()
    
    # Create a session for a user
    user_id = "user_123"
    session = TravelPlanningSession(user_id, storage, conversation_manager)
    
    # Start conversation
    print(session.start_session())
    print("\n" + "="*60 + "\n")
    
    # Simulate user interactions
    user_inputs = [
        "I'm thinking about Thailand!",
        "Starting March 15, 2026 and staying for 7 days",
        "My budget is around $2500",
        "I love beaches, food, and exploring local culture"
    ]
    
    for user_input in user_inputs:
        print(f"👤 User: {user_input}")
        response = session.process_message(user_input)
        print(f"🤖 Assistant: {response}\n")
        
        if session.conversation_manager.requirements.is_complete():
            print("\n✅ Ready to generate full plan!")
            break